# SQL 50 in Pandas

In [2]:
from typing import List
from typing import Optional
import pandas as pd
import numpy as np

### 1757. Recyclable and Low Fat Products [E]

Write a solution to find the ids of products that are both low fat and recyclable.

Return the result table in any order.

In [13]:
def find_products(products: pd.DataFrame) -> pd.DataFrame:
    return products[ (products['low_fats']=='Y') & (products['recyclable']=='Y') ][['product_id']]

In [2]:
data = [['0', 'Y', 'N'], ['1', 'Y', 'Y'], ['2', 'N', 'Y'], ['3', 'Y', 'Y'], ['4', 'N', 'N']]
products = pd.DataFrame(data, columns=['product_id', 'low_fats', 'recyclable']).astype({'product_id':'int64', 'low_fats':'category', 'recyclable':'category'})

In [14]:
find_products(products)

,product_id
1,1
3,3


### 2877. Create a DataFrame from List [E]

Write a solution to create a DataFrame from a 2D list called student_data. This 2D list contains the IDs and ages of some students.

The DataFrame should have two columns, student_id and age, and be in the same order as the original 2D list.

In [16]:
input_list = [
  [1, 15],
  [2, 11],
  [3, 11],
  [4, 20]
]

In [23]:
def createDataframe(student_data: List[List[int]]) -> pd.DataFrame:
    return pd.DataFrame(student_data, columns=['student_id','age'])

In [24]:
createDataframe(input_list)

,student_id,age
0,1,15
1,2,11
2,3,11
3,4,20


### 176. Second Highest Salary [M]

Write a solution to find the second highest distinct salary from the Employee table. If there is no second highest salary, return null (return None in Pandas).

The result format is in the following example.

In [64]:
def second_highest_salary(employee: pd.DataFrame) -> pd.DataFrame:
    df = employee['salary'].drop_duplicates().sort_values(ascending=False).reset_index(drop=True)
    if len(df) >= 2:
        return pd.DataFrame([df[1]], columns=['SecondHighestSalary'])
    else:
        return pd.DataFrame([None], columns=['SecondHighestSalary'])

In [25]:
data = [[1, 100], [2, 200], [3, 300]]
employee = pd.DataFrame(data, columns=['id', 'salary']).astype({'id':'int64', 'salary':'int64'})

In [67]:
second_highest_salary(employee)

,SecondHighestSalary
0,200


### 584. Find Customer Referee

Find the names of the customer that are not referred by the customer with id = 2.

Return the result table in any order.

In [105]:
def find_customer_referee(customer: pd.DataFrame) -> pd.DataFrame:
    return customer[ (customer['referee_id']!=2) | (customer['referee_id'].isnull() ) ][['name']] # .drop_duplicates()

In [80]:
data = [[1, 'Will', None], [2, 'Jane', None], [3, 'Alex', 2], [4, 'Bill', None], [5, 'Zack', 1], [6, 'Mark', 2]]
customer = pd.DataFrame(data, columns=['id', 'name', 'referee_id']).astype({'id':'Int64', 'name':'object', 'referee_id':'Int64'})

In [106]:
find_customer_referee(customer)

,name
0,Will
1,Jane
3,Bill
4,Zack
6,Tse


### 570. Managers with at Least 5 Direct Reports [M]

Write a solution to find managers with at least five direct reports.

Return the result table in any order.

In [129]:
def find_managers(employee: pd.DataFrame) -> pd.DataFrame:
    df5 = employee.groupby('managerId', as_index=False).agg({'id':'count'}).query('id >= 5').rename(columns={'id':'count'})
    if df5.shape[0] > 0:
        return pd.merge(left=employee, right=df5, how='inner', left_on='id', right_on='managerId')[['name']]
    else:
        return pd.DataFrame(None, columns=['name'])

In [107]:
data = [[101, 'John', 'A', None], [102, 'Dan', 'A', 101], [103, 'James', 'A', 101], [104, 'Amy', 'A', 101], [105, 'Anne', 'A', 101], [106, 'Ron', 'B', 101]]
employee = pd.DataFrame(data, columns=['id', 'name', 'department', 'managerId']).astype({'id':'Int64', 'name':'object', 'department':'object', 'managerId':'Int64'})

In [130]:
find_managers(employee)

,name


### 185. Department Top Three Salaries [H]

A company's executives are interested in seeing who earns the most money in each of the company's departments. A high earner in a department is an employee who has a salary in the top three unique salaries for that department.

Write a solution to find the employees who are high earners in each of the departments.

Return the result table in any order.

The result format is in the following example.

In [198]:
# Beats 69.03%

def top_three_salaries(employee: pd.DataFrame, department: pd.DataFrame) -> pd.DataFrame:
    department2 = department.rename(columns={'id':'departmentId','name':'Department'})
    employee2 = employee.rename(columns={'name':'Employee','salary':'Salary'})
    df = pd.merge(employee2, department2, how='left', on='departmentId') # id Employee salary departnemtnId Department

    if df.shape[0] < 1:
        return pd.DataFrame(None, columns=['Department','Employee','Salary'])

    count = 0
    for dept in department2['departmentId'].unique():
        df_loop = employee2[ employee2['departmentId']==dept ].sort_values('Salary', ascending=False)[['Salary','departmentId']].drop_duplicates().head(3)

        if count == 0:
            df_top3 = df_loop.copy()
        else:
            df_top3 = pd.concat([df_top3, df_loop], axis=0)
        count += 1

    df2 = pd.merge(df, df_top3, how='inner', on=['Salary', 'departmentId'])[['Department','Employee','Salary']]
    return df2

In [199]:
data = [[1, 'Joe', 85000, 1], [2, 'Henry', 80000, 2], [3, 'Sam', 60000, 2], [4, 'Max', 90000, 1], [5, 'Janet', 69000, 1], [6, 'Randy', 85000, 1], [7, 'Will', 70000, 1]]
employee = pd.DataFrame(data, columns=['id', 'name', 'salary', 'departmentId']).astype({'id':'Int64', 'name':'object', 'salary':'Int64', 'departmentId':'Int64'})
data = [[1, 'IT'], [2, 'Sales']]
department = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'Int64', 'name':'object'})

In [200]:
top_three_salaries(employee, department)

,Department,Employee,Salary
0,IT,Joe,85000
1,Sales,Henry,80000
2,Sales,Sam,60000
3,IT,Max,90000
4,IT,Randy,85000
5,IT,Will,70000


### 197. Rising Temperature [E]

Write a solution to find all dates' id with higher temperatures compared to its previous dates (yesterday).

Return the result table in any order.


In [248]:
def rising_temperature(weather: pd.DataFrame) -> pd.DataFrame:
    df = weather.sort_values('recordDate', ascending=True).reset_index(drop=True)
    df['day_diff'] = df['recordDate'].diff()
    df['temp_diff'] = df['temperature'].diff()
    return df[ (df['day_diff']== pd.Timedelta(days=1)) & (df['temp_diff']>0) ][['id']]

In [267]:
def rising_temperature(weather: pd.DataFrame) -> pd.DataFrame:
    weather['recordDate'] = pd.to_datetime(weather['recordDate'])
    weather2 = weather.copy()
    weather2['recordDate'] = weather2['recordDate'] + pd.Timedelta(days=1) # temp yesterday
    df = pd.merge(weather, weather2, how='left', on='recordDate') # x is today, y is yesterday
    return df[ df['temperature_x'] - df['temperature_y'] > 0 ][['id_x']].rename(columns={'id_x':'id'})


In [272]:
data = [[1, '2015-01-01', 10], [2, '2015-01-02', 25], [3, '2015-01-03', 20], [4, '2015-01-04', 30]]
# data = [[1, '2015-01-01', 10], [2, '2015-01-02', 25]]
weather = pd.DataFrame(data, columns=['id', 'recordDate', 'temperature']).astype({'id':'Int64', 'recordDate':'datetime64[ns]', 'temperature':'Int64'})

In [271]:
rising_temperature(weather)

,id
1,2


### 1148. Article Views I [E]

Write a solution to find all the authors that viewed at least one of their own articles.

Return the result table sorted by id in ascending order.

In [277]:
# Beats 7.45%
def article_views(views: pd.DataFrame) -> pd.DataFrame:
    return views[ views['author_id']==views['viewer_id'] ].groupby('author_id', as_index=False).agg({'article_id':'count'})[['author_id']].rename(columns={'author_id':'id'}).sort_values('id', ascending=True)

In [ ]:
# Beats 81.56
def article_views(views: pd.DataFrame) -> pd.DataFrame:
    return views.loc[ views['author_id']==views['viewer_id'] , ['author_id']].drop_duplicates().rename(columns={'author_id':'id'}).sort_values('id', ascending=True)

In [274]:
data = [[1, 3, 5, '2019-08-01'], [1, 3, 6, '2019-08-02'], [2, 7, 7, '2019-08-01'], [2, 7, 6, '2019-08-02'], [4, 7, 1, '2019-07-22'], [3, 4, 4, '2019-07-21'], [3, 4, 4, '2019-07-21']]
views = pd.DataFrame(data, columns=['article_id', 'author_id', 'viewer_id', 'view_date']).astype({'article_id':'Int64', 'author_id':'Int64', 'viewer_id':'Int64', 'view_date':'datetime64[ns]'})

In [ ]:
article_views(views)

,id
0,4
1,7


### 175. Combine Two Tables [E]

Write a solution to report the first name, last name, city, and state of each person in the Person table. If the address of a personId is not present in the Address table, report null instead.

Return the result table in any order.

In [284]:
# Beats 36.82%
def combine_two_tables(person: pd.DataFrame, address: pd.DataFrame) -> pd.DataFrame:
    # We want: first name, last name, city, state (for people in person table)
    return pd.merge(left=person, right=address, how='left', on='personId')[['firstName','lastName','city','state']]
    # personid firstName lastName addressId city state

In [280]:
data = [[1, 'Wang', 'Allen'], [2, 'Alice', 'Bob']]
person = pd.DataFrame(data, columns=['personId', 'firstName', 'lastName']).astype({'personId':'Int64', 'firstName':'object', 'lastName':'object'})
data = [[1, 2, 'New York City', 'New York'], [2, 3, 'Leetcode', 'California']]
address = pd.DataFrame(data, columns=['addressId', 'personId', 'city', 'state']).astype({'addressId':'Int64', 'personId':'Int64', 'city':'object', 'state':'object'})

In [285]:
combine_two_tables(person, address) 

,firstName,lastName,city,state
0,Wang,Allen,NaN,NaN
1,Alice,Bob,New York City,New York


### 181. Employees Earning More Than Their Managers [E]

Write a solution to find the employees who earn more than their managers.

Return the result table in any order.

In [453]:
# Beats 90.93%
def find_employees(employee: pd.DataFrame) -> pd.DataFrame:
    if employee.shape[0] < 1:
        return pd.DataFrame(None, columns=['Employee'])
    manager = employee.copy()
    manager = manager[['id','name','salary']].rename(columns = {'id':'managerId', 'name':'manager_name', 'salary':'manager_salary'}) # managerId manager_name manager_salary
    df = pd.merge(employee, manager, how='inner', on='managerId') # id name salary managerId manager_name manager_salary
    if df.shape[0] < 1:
        return pd.DataFrame(None, columns=['Employee'])
    else:
        return df[ df['salary'] > df['manager_salary'] ][['name']].rename(columns = {'name':'Employee'})

In [447]:
data = [[1, 'Joe', 70000, 3], [2, 'Henry', 80000, 4], [3, 'Sam', 60000, None], [4, 'Max', 90000, None]]
employee = pd.DataFrame(data, columns=['id', 'name', 'salary', 'managerId']).astype({'id':'Int64', 'name':'object', 'salary':'Int64', 'managerId':'Int64'})

In [454]:
find_employees(employee)

,Employee
0,Joe


### 1193. Monthly Transactions I [M]

Write an SQL query to find for each month and country, the number of transactions and their total amount, the number of approved transactions and their total amount.

Return the result table in any order.

In [306]:
def monthly_transactions(transactions: pd.DataFrame) -> pd.DataFrame:
    # transactions['month'] = pd.to_datetime(transactions['trans_date']).dt.to_period('M')
    # transactions.loc[ transactions['country'].isna(), transactions['country'] ] = 'null'

    transactions['month'] = pd.to_datetime(transactions['trans_date']).dt.strftime('%Y-%m')
    
    trans = transactions.groupby(['month','country'], as_index=False, dropna=False).agg(trans_count=('state','count'), trans_total_amount=('amount','sum'))
    trans_approved = transactions[ transactions['state']=='approved' ].groupby(['month','country'], as_index=False, dropna=False).agg(approved_count=('state','count'), approved_total_amount=('amount','sum'))
    
    if trans.shape[0] == 0:
        return pd.DataFrame(None, columns=['month','country','trans_count','approved_count','trans_total_amount','approved_total_amount'])
    
    df = pd.merge(trans, trans_approved, how='left', on=['month','country'])[['month','country','trans_count','approved_count','trans_total_amount','approved_total_amount']]

    # Fix approved ds NAs if exist
    df.loc[ df['approved_count'].isna(), ['approved_count'] ] = 0
    df.loc[ df['approved_total_amount'].isna(), ['approved_total_amount'] ] = 0

    return df

In [55]:
data = [[121, 'US', 'approved', 1000, '2018-12-18'], [122, 'US', 'declined', 2000, '2018-12-19'], [123, 'US', 'approved', 2000, '2019-01-01'], [124, 'DE', 'approved', 2000, '2019-01-07']]
# data = [[121, 'US', 'declined', 1000, '2018-12-18'], [122, 'US', 'declined', 2000, '2018-12-19'], [123, 'US', 'declined', 2000, '2019-01-01'], [124, 'DE', 'declined', 2000, '2019-01-07']]
transactions = pd.DataFrame(data, columns=['id', 'country', 'state', 'amount', 'trans_date']).astype({'id':'Int64', 'country':'object', 'state':'object', 'amount':'Int64', 'trans_date':'datetime64[ns]'})

In [305]:
monthly_transactions(transactions)

,month,country,trans_count,approved_count,trans_total_amount,approved_total_amount
0,2018-12,US,2,0.0,3000,0
1,2019-01,DE,1,0.0,2000,0
2,2019-01,US,1,0.0,2000,0


### 595. Big Countries [E]

A country is big if:

* it has an area of at least three million (i.e., 3000000 km2), or

* it has a population of at least twenty-five million (i.e., 25000000).

Write a solution to find the name, population, and area of the big countries.

Return the result table in any order.

In [310]:
def big_countries(world: pd.DataFrame) -> pd.DataFrame:
    return world[ (world['area'] >= 3000000) | (world['population'] >= 25000000) ][['name','population','area']]

In [307]:
data = [['Afghanistan', 'Asia', 652230, 25500100, 20343000000], ['Albania', 'Europe', 28748, 2831741, 12960000000], ['Algeria', 'Africa', 2381741, 37100000, 188681000000], ['Andorra', 'Europe', 468, 78115, 3712000000], ['Angola', 'Africa', 1246700, 20609294, 100990000000]]
world = pd.DataFrame(data, columns=['name', 'continent', 'area', 'population', 'gdp']).astype({'name':'object', 'continent':'object', 'area':'Int64', 'population':'Int64', 'gdp':'Int64'})

In [311]:
big_countries(world)

,name,population,area
0,Afghanistan,25500100,652230
2,Algeria,37100000,2381741


### 1661. Average Time of Process per Machine [E]

There is a factory website that has several machines each running the same number of processes. Write a solution to find the average time each machine takes to complete a process.

The time to complete a process is the 'end' timestamp minus the 'start' timestamp. The average time is calculated by the total time to complete every process on the machine divided by the number of processes that were run.

The resulting table should have the machine_id along with the average time as processing_time, which should be rounded to 3 decimal places.

Return the result table in any order.


In [319]:
def get_average_time(activity: pd.DataFrame) -> pd.DataFrame:
    # machine_id processing_time (rounded to 3)
    # processing_time = total_time / #_processes

    if activity.shape[0] == 0:
        return pd.DataFrame(None, columns=['machine_id','processing_time'])

    activity_start = activity[ activity['activity_type'] == 'start' ][['machine_id','process_id','timestamp']].rename(columns={'timestamp':'time_start'})
    activity_end = activity[ activity['activity_type'] == 'end' ][['machine_id','process_id','timestamp']].rename(columns={'timestamp':'time_end'})

    df = pd.merge(activity_start, activity_end, how='left', on=['machine_id','process_id']) # machine_id process_id time_start time_end

    df['process_time'] = df['time_end'] - df['time_start']  # machine_id process_id time_start time_end process_time

    df_grouped = df.groupby(['machine_id'], as_index=False).agg(process_count=('process_id','count'), process_total_time=('process_time','sum')) # machine_id process_count process_total_time

    df_grouped['processing_time'] = round(df_grouped['process_total_time']/df_grouped['process_count'] + 1e-9, 3)

    return df_grouped[['machine_id','processing_time']]
    

In [312]:
data = [[0, 0, 'start', 0.712], [0, 0, 'end', 1.52], [0, 1, 'start', 3.14], [0, 1, 'end', 4.12], [1, 0, 'start', 0.55], [1, 0, 'end', 1.55], [1, 1, 'start', 0.43], [1, 1, 'end', 1.42], [2, 0, 'start', 4.1], [2, 0, 'end', 4.512], [2, 1, 'start', 2.5], [2, 1, 'end', 5]]
activity = pd.DataFrame(data, columns=['machine_id', 'process_id', 'activity_type', 'timestamp']).astype({'machine_id':'Int64', 'process_id':'Int64', 'activity_type':'object', 'timestamp':'Float64'})

In [320]:
get_average_time(activity)

,machine_id,processing_time
0,0,0.894
1,1,0.995
2,2,1.456


### 1378. Replace Employee ID With The Unique Identifier [E]

Write a solution to show the unique ID of each user, If a user does not have a unique ID just show null.

Return the result table in any order.

In [333]:
def replace_employee_id(employees: pd.DataFrame, employee_uni: pd.DataFrame) -> pd.DataFrame:
    if employees.shape[0] == 0:
        return pd.DataFrame(None, columns=['unique_id','name'])
    df = pd.merge(employees, employee_uni, how='left', on='id')[['unique_id','name']] # use how='left' in case employee_uni is empty
    return df

In [325]:
data = [[1, 'Alice'], [7, 'Bob'], [11, 'Meir'], [90, 'Winston'], [3, 'Jonathan']]
employees = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'int64', 'name':'object'})
data = [[3, 1], [11, 2], [90, 3]]
employee_uni = pd.DataFrame(data, columns=['id', 'unique_id']).astype({'id':'int64', 'unique_id':'int64'})

In [334]:
replace_employee_id(employees, employee_uni)

,unique_id,name
0,NaN,Alice
1,NaN,Bob
2,2.0,Meir
3,3.0,Winston
4,1.0,Jonathan


### 1581. Customer Who Visited but Did Not Make Any Transactions [E]

Write a solution to find the IDs of the users who visited without making any transactions and the number of times they made these types of visits.

Return the result table sorted in any order.

In [464]:
# Beats 56.20%
def find_customers(visits: pd.DataFrame, transactions: pd.DataFrame) -> pd.DataFrame:
    # customer_id count_no_trans
    if visits.shape[0] ==0:
        return pd.DataFrame(None, columns=['customer_id','count_no_trans'])

    df = pd.merge(visits, transactions, how='left', on='visit_id') # visit_id customer_id transaction_id amount

    if df.shape[0] == 0:
        return pd.DataFrame(None, columns=['customer_id','count_no_trans'])
    else:
        return df[ df['transaction_id'].isna() ][['visit_id','customer_id']].groupby('customer_id', as_index=False).agg(count_no_trans=('visit_id','count'))

In [462]:
data = [[1, 23], [2, 9], [4, 30], [5, 54], [6, 96], [7, 54], [8, 54]]
visits = pd.DataFrame(data, columns=['visit_id', 'customer_id']).astype({'visit_id':'Int64', 'customer_id':'Int64'})
data = [[2, 5, 310], [3, 5, 300], [9, 5, 200], [12, 1, 910], [13, 2, 970]]
transactions = pd.DataFrame(data, columns=['transaction_id', 'visit_id', 'amount']).astype({'transaction_id':'Int64', 'visit_id':'Int64', 'amount':'Int64'})

In [465]:
find_customers(visits, transactions)

,customer_id,count_no_trans
0,30,1
1,54,2
2,96,1


### 1164. Product Price at a Given Date [M]

Write a solution to find the prices of all products on 2019-08-16. Assume the price of all products before any change is 10.

Return the result table in any order.

In [481]:
# Beats 16.21%
def price_at_given_date(products: pd.DataFrame) -> pd.DataFrame:
    # date 2019-08-16

    valid_changes = products[ products['change_date'] <= '2019-08-16' ] # changes before cutoff date

    # this is product-date pairings that are before, closest to cutoff
    valid_changes = valid_changes.groupby('product_id').agg(change_date=('change_date','max')) # product_id change_date
    valid_changes['valid'] = 1 # add an indicator for merge

    valid_changes_price = pd.merge(products, valid_changes, how='left', on=['product_id','change_date']) # product_id new_price change_date valid
    valid_changes_price = valid_changes_price[ valid_changes_price['valid'] == 1 ][['product_id','new_price']]

    total_products = products[['product_id']].drop_duplicates()

    final_df = pd.merge(total_products, valid_changes_price, how='left', on='product_id').rename(columns={'new_price':'price'}) # product_id price

    final_df.loc[ final_df['price'].isna(), ['price'] ] = 10 # set missing prices to 10

    return final_df

    # typos: date, datseset name ('products' vs 'product_id'), missing an 's' on 'valid_changes'

In [57]:
data = [[1, 20, '2019-08-14'], [2, 50, '2019-08-14'], [1, 30, '2019-08-15'], [1, 35, '2019-08-16'], [2, 65, '2019-08-17'], [3, 20, '2019-08-18']]
products = pd.DataFrame(data, columns=['product_id', 'new_price', 'change_date']).astype({'product_id':'Int64', 'new_price':'Int64', 'change_date':'datetime64[ns]'})

### 1280. Students and Examinations [E]

Write a solution to find the number of times each student attended each exam.

Return the result table ordered by student_id and subject_name.

In [ ]:
# Beats 27.73%
def students_and_examinations(students: pd.DataFrame, subjects: pd.DataFrame, examinations: pd.DataFrame) -> pd.DataFrame:
    # student_id student_name subject_name attended_exams
    # order by student_id subject_name
    if students.shape[0]==0 or subjects.shape[0]==0 or examinations.shape[0]==0:
        return pd.DataFrame(None, columns=['student_id','student_name','subject_name','attended_exams'])

    student_subject = pd.merge(students, subjects, how='cross') # student_id student_name subject_name
    examinations['count'] = 1
    exams_grouped = examinations.groupby(['student_id','subject_name'], as_index=False).agg(attended_exams=('count','count')) # student_id subject_name attended_exams
    df = pd.merge(student_subject, exams_grouped, how='left', on=['student_id','subject_name'])
    df.loc[ df['attended_exams'].isna(), ['attended_exams'] ] = 0    # !!!!!!
    return df.sort_values(['student_id','subject_name'], ascending=True)

In [483]:
data = [[1, 'Alice'], [2, 'Bob'], [13, 'John'], [6, 'Alex']]
students = pd.DataFrame(data, columns=['student_id', 'student_name']).astype({'student_id':'Int64', 'student_name':'object'})
data = [['Math'], ['Physics'], ['Programming']]
subjects = pd.DataFrame(data, columns=['subject_name']).astype({'subject_name':'object'})
data = [[1, 'Math'], [1, 'Physics'], [1, 'Programming'], [2, 'Programming'], [1, 'Physics'], [1, 'Math'], [13, 'Math'], [13, 'Programming'], [13, 'Physics'], [2, 'Math'], [1, 'Math']]
examinations = pd.DataFrame(data, columns=['student_id', 'subject_name']).astype({'student_id':'Int64', 'subject_name':'object'})

In [500]:
students_and_examinations(students, subjects, examinations)

,student_id,student_name,subject_name,attended_exams
0,1,Alice,Math,3.0
1,1,Alice,Physics,2.0
2,1,Alice,Programming,1.0
3,2,Bob,Math,1.0
4,2,Bob,Physics,0.0
5,2,Bob,Programming,1.0
9,6,Alex,Math,0.0
10,6,Alex,Physics,0.0
11,6,Alex,Programming,0.0
6,13,John,Math,1.0


### 1934. Confirmation Rate [M]

The confirmation rate of a user is the number of 'confirmed' messages divided by the total number of requested confirmation messages. The confirmation rate of a user that did not request any confirmation messages is 0. Round the confirmation rate to two decimal places.

Write a solution to find the confirmation rate of each user.

Return the result table in any order.

In [34]:
# Beats 74.57%
def confirmation_rate(signups: pd.DataFrame, confirmations: pd.DataFrame) -> pd.DataFrame:
    # confirmed messages / total number of requested
    # 0 for missings (no requests)
    # round(x, 2)
    if signups.shape[0] == 0:
        return pd.DataFrame(None, columns=['user_id','confirmation_rate'])
    
    user_totals = confirmations.groupby('user_id', as_index=False).agg(count_total=('action','count')) # user_id action
    user_confirm =  confirmations[ confirmations['action'] == 'confirmed' ].groupby('user_id', as_index=False).agg(count_confirm=('action','count'))
    user_confirm_totals = pd.merge(user_totals, user_confirm, how='left', on='user_id')
    user_confirm_totals['confirmation_rate'] = round(user_confirm_totals['count_confirm']/user_confirm_totals['count_total'] + 1e-9, 2)
    df = pd.merge(signups, user_confirm_totals, how='left', on='user_id')[['user_id','confirmation_rate']] # user_id  (with missings for action)
    # df = df.loc[ df['confirmation_rate'].isna(), 'confirmation_rate' ] = 0
    df['confirmation_rate'] = df['confirmation_rate'].fillna(0)
    return df

In [29]:
data = [[3, '2020-03-21 10:16:13'], [7, '2020-01-04 13:57:59'], [2, '2020-07-29 23:09:44'], [6, '2020-12-09 10:39:37']]
signups = pd.DataFrame(data, columns=['user_id', 'time_stamp']).astype({'user_id':'Int64', 'time_stamp':'datetime64[ns]'})
data = [[3, '2021-01-06 03:30:46', 'timeout'], [3, '2021-07-14 14:00:00', 'timeout'], [7, '2021-06-12 11:57:29', 'confirmed'], [7, '2021-06-13 12:58:28', 'confirmed'], [7, '2021-06-14 13:59:27', 'confirmed'], [2, '2021-01-22 00:00:00', 'confirmed'], [2, '2021-02-28 23:59:59', 'timeout']]
confirmations = pd.DataFrame(data, columns=['user_id', 'time_stamp', 'action']).astype({'user_id':'Int64', 'time_stamp':'datetime64[ns]', 'action':'object'})

In [35]:
confirmation_rate(signups, confirmations)

,user_id,confirmation_rate
0,3,0.0
1,7,1.0
2,2,0.5
3,6,0.0


### 1251. Average Selling Price [E]

Write a solution to find the average selling price for each product. average_price should be rounded to 2 decimal places. If a product does not have any sold units, its average selling price is assumed to be 0.

Return the result table in any order.

In [15]:
# Beats 40.31%
def average_selling_price(prices: pd.DataFrame, units_sold: pd.DataFrame) -> pd.DataFrame:
    if units_sold.shape[0] == 0:
        df = prices[['product_id']].drop_duplicates()
        df['average_price'] = 0
        return df
    df = pd.merge(prices, units_sold, how='left', on='product_id') # product_id start end price purchase_date units
    df = df[ (df['purchase_date'] >= df['start_date']) & (df['purchase_date'] <= df['end_date']) ]
    df['price_unit'] = df['price'] * df['units'] # product_id unit price price_unit   sum: price_unit unit
    df = df.groupby(['product_id'], as_index=False).agg(price_unit=('price_unit','sum'), unit=('units','sum'))
    df['average_price'] = round(df['price_unit']/df['unit'] + 1e-9, 2)
    df = df[['product_id','average_price']]

    prod_master = prices[['product_id']].drop_duplicates()
    df = pd.merge(prod_master, df, how='left', on='product_id')
    df['average_price'] = df['average_price'].fillna(0)
    return df

In [3]:
data = [[1, '2019-02-17', '2019-02-28', 5], [1, '2019-03-01', '2019-03-22', 20], [2, '2019-02-01', '2019-02-20', 15], [2, '2019-02-21', '2019-03-31', 30]]
prices = pd.DataFrame(data, columns=['product_id', 'start_date', 'end_date', 'price']).astype({'product_id':'Int64', 'start_date':'datetime64[ns]', 'end_date':'datetime64[ns]', 'price':'Int64'})
data = [[1, '2019-02-25', 100], [1, '2019-03-01', 15], [2, '2019-02-10', 200], [2, '2019-03-22', 30]]
units_sold = pd.DataFrame(data, columns=['product_id', 'purchase_date', 'units']).astype({'product_id':'Int64', 'purchase_date':'datetime64[ns]', 'units':'Int64'})

In [14]:
df = pd.merge(prices, units_sold, how='left', on='product_id') # product_id start end price purchase_date units
df = df[ (df['purchase_date'] >= df['start_date']) & (df['purchase_date'] <= df['end_date']) ]
df['price_unit'] = df['price'] * df['units'] # product_id unit price price_unit   sum: price_unit unit
df = df.groupby(['product_id'], as_index=False).agg(price_unit=('price_unit','sum'), unit=('units','sum'))
df['average_price'] = round(df['price_unit']/df['unit'] + 1e-9, 2)
df = df[['product_id','average_price']]

prod_master = units_sold[['product_id']].drop_duplicates()
df = pd.merge(prod_master, df, how='left', on='product_id')
df['average_price'] = df['average_price'].fillna(0)
df


,product_id,average_price
0,1,6.96
1,2,16.96


### 180. Consecutive Numbers [M]

Find all numbers that appear at least three times consecutively.

Return the result table in any order.

In [51]:
# Beats 37.88%
def consecutive_numbers(logs: pd.DataFrame) -> pd.DataFrame:
    if logs.shape[0] < 3:
        return pd.DataFrame(None, columns=(['ConsecutiveNums']))
    logs['diff'] = [0] + [0] + [1 if ( logs['num'][i]==logs['num'][i-1] ) & ( logs['num'][i]==logs['num'][i-2] ) else 0 for i in range(2,logs.shape[0])]
    return logs.loc[ logs['diff']==1, ['num'] ].drop_duplicates().rename(columns={'num':'ConsecutiveNums'})

In [49]:
data = [[1, 1], [2, 1], [3, 1], [4, 2], [5, 1], [6, 2], [7, 2]]
logs = pd.DataFrame(data, columns=['id', 'num']).astype({'id':'Int64', 'num':'Int64'})

In [52]:
consecutive_numbers(logs)

,ConscutiveNums
2,1


In [54]:
pd.DataFrame(None, columns=(['ConsecutiveNums']))

,ConsecutiveNums


In [58]:
products

,product_id,new_price,change_date
0,1,20,2019-08-14
1,2,50,2019-08-14
2,1,30,2019-08-15
3,1,35,2019-08-16
4,2,65,2019-08-17
5,3,20,2019-08-18


### 626. Exchange Seats [M]

Write a solution to swap the seat id of every two consecutive students. If the number of students is odd, the id of the last student is not swapped.

Return the result table ordered by id in ascending order.


In [129]:
# Beats 47.49%
def exchange_seats(seat: pd.DataFrame) -> pd.DataFrame:
    if seat.shape[0] < 2:
        return seat

    seat = seat.sort_values(['id'], ascending=True)
    seat = seat.rename(columns={'id':'id_old'})

    if seat.shape[0]%2==1: # if odd
        seat['id'] = [ seat['id_old'][i+1] if seat['id_old'][i]%2 == 1 else seat['id_old'][i-1] for i in range(0,seat.shape[0]-1)] + [seat['id_old'][seat.shape[0]-1]]
    else:
        seat['id'] = [ seat['id_old'][i+1] if seat['id_old'][i]%2 == 1 else seat['id_old'][i-1] for i in range(0,seat.shape[0])]

    return seat[['id','student']].sort_values(['id'], ascending=True).reset_index(drop=True)

In [121]:
data = [[1, 'Abbot'], [2, 'Doris'], [3, 'Emerson'], [4, 'Green'], [5, 'Jeames']]
seat = pd.DataFrame(data, columns=['id', 'student']).astype({'id':'Int64', 'student':'object'})

In [130]:
exchange_seats(seat)

,id,student
0,1,Doris
1,2,Abbot
2,3,Green
3,4,Emerson
4,5,Jeames


### 577. Employee Bonus [E]

Write a solution to report the name and bonus amount of each employee with a bonus less than 1000.

Return the result table in any order.

In [140]:
def employee_bonus(employee: pd.DataFrame, bonus: pd.DataFrame) -> pd.DataFrame:
    # name, amount for bonus < 1000
    df = pd.merge(employee, bonus, how='left', on='empId')
    df = df[ (df['bonus'] < 1000) | (df['bonus'].isna()) ]
    return df[['name','bonus']]

In [131]:
data = [[3, 'Brad', None, 4000], [1, 'John', 3, 1000], [2, 'Dan', 3, 2000], [4, 'Thomas', 3, 4000]]
employee = pd.DataFrame(data, columns=['empId', 'name', 'supervisor', 'salary']).astype({'empId':'Int64', 'name':'object', 'supervisor':'Int64', 'salary':'Int64'})
data = [[2, 500], [4, 2000]]
bonus = pd.DataFrame(data, columns=['empId', 'bonus']).astype({'empId':'Int64', 'bonus':'Int64'})

In [141]:
employee_bonus(employee, bonus)

,name,bonus
0,Brad,<NA>
1,John,<NA>
2,Dan,500


### 610. Triangle Judgement [E]

Report for every three line segments whether they can form a triangle.

Return the result table in any order.

In [144]:
# Beats 30.33%
def triangle_judgement(triangle: pd.DataFrame) -> pd.DataFrame:
    # max < sum total  - max     (sum other sides)
    # 0 < sum total - 2 max
    triangle['triangle'] = np.where(triangle.sum(axis=1) - 2* triangle.max(axis=1) > 0, 'Yes', 'No')
    return triangle


In [142]:
data = [[13, 15, 30], [10, 20, 15]]
triangle = pd.DataFrame(data, columns=['x', 'y', 'z']).astype({'x':'Int64', 'y':'Int64', 'z':'Int64'})

In [145]:
triangle_judgement(triangle)

,x,y,z,triangle
0,13,15,30,No
1,10,20,15,Yes


### 1204. Last Person to Fit in the Bus [M]

There is a queue of people waiting to board a bus. However, the bus has a weight limit of 1000 kilograms, so there may be some people who cannot board.

Write a solution to find the person_name of the last person that can fit on the bus without exceeding the weight limit. The test cases are generated such that the first person does not exceed the weight limit.

Note that only one person can board the bus at any given turn.

In [153]:
# Beats 5.05%
def last_passenger(queue: pd.DataFrame) -> pd.DataFrame:
    queue = queue.sort_values(['turn'], ascending=True).reset_index(drop=True)
    cweight = 0
    queue['cweight'] = 0
    for i in range(0,queue.shape[0]):
        cweight += queue['weight'][i]
        queue.loc[ i, ['cweight'] ] = cweight

    df = queue[ queue['cweight'] <= 1000 ].tail(1)
    return df[['person_name']]

In [146]:
data = [[5, 'Alice', 250, 1], [4, 'Bob', 175, 5], [3, 'Alex', 350, 2], [6, 'John Cena', 400, 3], [1, 'Winston', 500, 6], [2, 'Marie', 200, 4]]
queue = pd.DataFrame(data, columns=['person_id', 'person_name', 'weight', 'turn']).astype({'person_id':'Int64', 'person_name':'object', 'weight':'Int64', 'turn':'Int64'})

In [154]:
last_passenger(queue)

,person_name
2,John Cena


### 50. Game Play Analysis IV [M]

Write a solution to report the fraction of players that logged in again on the day after the day they first logged in, rounded to 2 decimal places. In other words, you need to count the number of players that logged in for at least two consecutive days starting from their first login date, then divide that number by the total number of players.

In [192]:
# Beats 18.91%
def gameplay_analysis(activity: pd.DataFrame) -> pd.DataFrame:
    first = activity.groupby(['player_id']).agg(first_date=('event_date','min'))
    first['second_date'] = first['first_date'] + pd.Timedelta(days=1)
    df = pd.merge(activity, first, left_on=['player_id','event_date'], right_on=['player_id','second_date'])
    count_seond_login = df.shape[0]
    count_total_login = len(activity['player_id'].unique())
    est = round(count_seond_login/count_total_login,2)
    return pd.DataFrame([est], columns=['fraction'])

In [155]:
data = [[1, 2, '2016-03-01', 5], [1, 2, '2016-03-02', 6], [2, 3, '2017-06-25', 1], [3, 1, '2016-03-02', 0], [3, 4, '2018-07-03', 5]]
activity = pd.DataFrame(data, columns=['player_id', 'device_id', 'event_date', 'games_played']).astype({'player_id':'Int64', 'device_id':'Int64', 'event_date':'datetime64[ns]', 'games_played':'Int64'})

In [191]:
gameplay_analysis(activity)

,fraction
0,0.33


## Other: Second transaction by group

Assume you are given the table below on Uber transactions made by users. Write a query to obtain the third transaction of every user. Output the user id, spend and transaction date.

https://datalemur.com/questions/sql-third-transaction

In [65]:
products.groupby(['product_id'], as_index=False).agg(second_date=('change_date',lambda x: x.sort_values().head(2).tail(1)))

,product_id,second_date
0,1,2019-08-15
1,2,2019-08-17
2,3,2019-08-18
